# Smart-G demo notebook

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%load_ext autoreload
%autoreload 2
from smartg import RoughSurface, LambSurface, FlatSurface, Environment
from smartgx import smartg, smartg_thr, reptran_merge
from PAR import SpecInt, SpecInt2, Irr, ReadREPTRAN_bands
from profile.profil import Profile, AeroOPAC, CloudOPAC
from luts import LUT, MLUT, Idx, merge, read_lut_hdf, read_mlut_hdf
from water.iop_spm import IOP_SPM
from water.iop_mm import IOP_MM
from smartg_view import semi_polar, smartg_view
from IPython.html.widgets.interaction import interact, interact_manual

## How to run Smart-G

In [ ]:
# run SMART-G
# this simulation uses an AFGL tropical atmosphere without aerosols
m = smartg(THVDEG=30, wl=500., NBPHOTONS=1e8,
           atm=Profile('afglt'),
           surf=RoughSurface(SUR=1))

In [ ]:
# the result is a MLUT
m.print_info()
# it can be written with m.save(filename)
# and then reloaded with read_mlut_hdf(filename)

## How to view the results

In [ ]:
# basic visualization
smartg_view(m)

In [ ]:
# access a LUT within the MLUT
I = m['I_up (TOA)']  # or by index: m[0]

In [ ]:
I.print_info()

In [ ]:
# access values in this LUT
print I[5, ::5]  # by index (can be float indices to interpolate)
print I[Idx(90.), Idx(45.)]  # or by index finding + interpolation

In [ ]:
# calculate polarized light
# use operations between LUT's, and apply sqrt
Q, U = m['Q_up (TOA)'], m['U_up (TOA)']
LP = (Q*Q + U*U).apply(sqrt, 'LP')   # second parameter of apply sets the LUT's desc
LP.plot()

In [ ]:
I.plot(index=Idx(45), vmin=0, vmax=0.3)   # the arguments are optional

In [ ]:
# get a sub-LUT from a LUT (here 2D -> 1D)
I.sub()[Idx(90),:].plot()

In [ ]:
# interactive display
interact_manual(lambda vmax, phi:
                I.plot(index=Idx(phi), vmin=0, vmax=vmax),
                vmax=(0,1.,0.01),
                phi=(0, 180),
               )

## A few examples

In [ ]:
# Basic Rayleigh example, no surface
smartg_view(smartg(wl=500., THVDEG=30., NBPHOTONS=1e8, atm=Profile('afglt')))

In [ ]:
# only Rayleigh with a custom grid
smartg(wl=500., DEPO=0., NBPHOTONS=1e9,
        atm=Profile('afglt', grid='100[75]25[5]10[1]0'))

In [ ]:
# Rayleigh + aerosols
aer = AeroOPAC('urban', 0.4, 550.)
pro = Profile('afglms', aer=aer)
smartg(wl=300., THVDEG=60, atm=pro, NBPHOTONS=1e8)

In [ ]:
# atmosphere + surface
smartg(490., NBPHOTONS=1e7, THVDEG=45.,
        atm=Profile('afglms'),
        surf=LambSurface(ALB=0.1))

In [ ]:
# atmosphere + surface + océan
smartg(490., NBPHOTONS=1e7, THVDEG=30.,
            atm=Profile('afglms'),
            surf=RoughSurface(),
            water=IOP_MM(1.))

In [ ]:
# surface + océan (SPM model)
smartg(750., THVDEG=30., NBPHOTONS=1e9,
        surf=RoughSurface(),
        water=IOP_SPM(SPM=100.))

In [ ]:
# océan seul (chl model)
smartg(wl=750., THVDEG=30., water=IOP_SPM(1.), NBPHOTONS=1e9)

## Multispectral

In [ ]:
# multispectral simulation
pro = Profile('afglt',
              grid=[100, 75, 50, 30, 20, 10, 5, 1, 0.],  # optional, otherwise use default grid
              pfgrid=[100, 20, 0.],   # optional, otherwise use a single band 100-0
              pfwav=[400, 500, 600], # optional, otherwise phase functions are calculated at all bands
              aer=AeroOPAC('maritime_clean', 0.3, 550.),
              verbose=True)
m = smartg(wl=np.linspace(400, 600, 10.),
           THVDEG=60.,
           atm=pro,
           surf=RoughSurface(),
           water=IOP_SPM(1.))
m['I_up (TOA)'].sub()[0,:,:].plot()
figure()
m['I_up (TOA)'].sub()[:,Idx(30),Idx(30)].plot()

## REPTRAN

### Example 1

In [ ]:
# REPTRAN (example 1)
repname = 'reptran_solar_msg' # k distribution file, here MSG/SEVIRI solar channels
BAND = 'msg1_seviri_ch006'
surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34)
atm=Profile('afglmw')  
# Read Reptran band characteristics
wi, wb, we, ex, dl, bands = ReadREPTRAN_bands(repname, BAND=BAND, FULL=True)
M2 = smartg(THVDEG=30, wl=bands, NBPHOTONS=1e8, atm=atm, surf=surf)

In [ ]:
print '--M-----------',M2['I_up (TOA)'].print_info()
fig=figure(figsize=(8,4))
semi_polar(M2['I_up (TOA)'].sub()[0,:,:],rect=121,fig=fig,vmin=0,vmax=0.2)
semi_polar(M2['I_up (TOA)'].sub()[1,:,:],rect=122,fig=fig,vmin=0,vmax=0.2)
s = SpecInt2(wi,wb,ex,we,dl,M=M2,field='I_up (TOA)')
print '--S-----------',s.print_info()
fig2=figure(figsize=(4,4))
semi_polar(s.sub()[0,:,:], vmin=0, vmax=0.2, rect=111, fig=fig2)

In [ ]:
M=job.read()
MM_l=[]
for k in range(len(M.luts)):
    MM_tmp = M.luts[k]
    MM=LUT(MM_tmp.data,axes=[wi.data,MM_tmp.axes[1],MM_tmp.axes[2]],desc=MM_tmp.desc,
           names=['Wavelengths',MM_tmp.names[1],MM_tmp.names[2]],attrs=MM_tmp.attrs)
    MM_l.append(MM)
M2 = MLUT(MM_l)
print '--M-----------',M2['I_up (TOA)'].print_info()
fig=figure(figsize=(8,4))
semi_polar(M2['I_up (TOA)'].sub[0,:,:],rect=121,fig=fig,vmin=0,vmax=0.2)
semi_polar(M2['I_up (TOA)'].sub[1,:,:],rect=122,fig=fig,vmin=0,vmax=0.2)
s = SpecInt2(wi,wb,ex,we,dl,M=M2,field='I_up (TOA)')
print '--S-----------',s.print_info()
fig2=figure(figsize=(4,4))
semi_polar(s.sub[0,:,:],vmin=0,vmax=0.2,rect=111,fig=fig2)

### Example 2

In [ ]:
# REPTRAN (example 2)
repname = 'reptran_solar_medium' # k distribution file, here full solar channels at mdeium resolution
LMIN=765. # nm within O2A bands
LMAX=768. # nm
SAMPLING=1 # sampling of individual channels 1: all microchannels
surf=RoughSurface(SUR=3, WIND=5., NH2O=1.34)
atm=Profile('afglmw')  
# Read Reptran band characteristics
wi,wb,we,ex,dl,bands = ReadREPTRAN_bands(repname, LMIN=LMIN, LMAX=LMAX, SAMPLING=SAMPLING, FULL=True)
M2 = smartg(THVDEG=30, wl=bands, NBPHOTONS=1e9, atm=atm, surf=surf)

In [ ]:
print '--M-----------',M2['I_up (TOA)'].print_info()
fig=figure(figsize=(8,4))
semi_polar(M2['I_up (TOA)'].sub()[Idx(765.5),:,:],rect=121,fig=fig,vmin=0,vmax=0.1)
semi_polar(M2['I_up (TOA)'].sub()[Idx(767.5),:,:],rect=122,fig=fig,vmin=0,vmax=0.1)
s = SpecInt2(wi,wb,ex,we,dl,M=M2,field='I_up (TOA)')
print '--S-----------',s.print_info()
fig2=figure(figsize=(8,4))
semi_polar(s.sub()[Idx(765.5),:,:],vmin=0,vmax=0.1,sub=223,index=Idx(140),rect=221,fig=fig2)
semi_polar(s.sub()[:,Idx(140),:], rect=222, sub=224, index=Idx(30),swap=True,fig=fig2,vmin=0,vmax=0.1)

## Looping over parameters

In [ ]:
# loop over some wavelengths
MLUTS = []
for wl in linspace(400, 800, 10):
    M = smartg(THVDEG=30,
               wl=wl, NBPHOTONS=1e7,
               atm=Profile('afglt'),
               surf=RoughSurface())
    MLUTS.append(M)

In [ ]:
M = merge(MLUTS, ['Wavelength'], dtype=float)

In [ ]:
M[0].sub()[0,:,:].plot(index=Idx(45))

## Difference between two LUTs

In [ ]:
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
results = []
for pp in [True, False]:
    results.append(smartg(pp=pp, THVDEG=60, wl=443., NBPHOTONS=1e10,
                          NBTHETA=20., NBPHI=20.,
                          atm=Profile('afglt')))
    
PP = results[0]['I_up (TOA)']
SP = results[1]['I_up (TOA)']

# show relative difference
semi_polar(100.*(SP-PP)/PP, Idx(175.), fig=figure(figsize(8, 8)), vmin=-2, vmax=2)

In [ ]:
semi_polar(SP/PP, Idx(90.), fig=figure(figsize(8, 8)),vmin=0.97,vmax=1.03)

## Interactive simulation

In [ ]:
from IPython.html.widgets.interaction import interact, interact_manual

def simulate(thvdeg, surface, aerosol_model, aot550, wavelength):
    surf = {True:RoughSurface(), False:None}[surface]
    aer = AeroOPAC(aerosol_model, aot550, 550.)
    smartg_view(smartg(THVDEG=thvdeg,
             wl=wavelength, NBPHOTONS=1e7,
             NBTHETA=90, NBPHI=90,
             atm=Profile('afglt', aer=aer), surf=surf, OUTPUT_LAYERS=3
          ), QU=False, field='up (0+)')

interact_manual(simulate, thvdeg=(0, 90), surface=True,
                aerosol_model=AeroOPAC.listStandardAerosolFiles(), aot550=(0.001, 5., 0.01),
                wavelength=(400., 800.))